In [64]:
import numpy as np
import functools as ft
import operator as opt

In [86]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Wire:
    def __init__(self, x_1, y_1, x_2, y_2, steps = 0):
        self.start = Point(x_1, y_1)
        self.end = Point(x_2, y_2)
        self.steps = steps
    def set_steps(self, steps):
        self.steps = steps

In [102]:
data = np.loadtxt("03_data.txt",delimiter=",", dtype="str")

In [103]:
str = "123"
print(str[0:len(str)])

123


In [104]:
test_data = np.loadtxt("03_test_data.txt",delimiter=",", dtype="str")

In [105]:
print(test_data)

[['R8' 'U5' 'L5' 'D3']
 ['U7' 'R6' 'D4' 'L4']]


In [71]:
def get_wire_as_section(wire):
    x = 0
    y = 0
    sections = []
    for move in wire:
        if (move[0] == 'L'):
            distance = int(move[1:len(move)])
            sections.append(((x,y), (x-distance, y)))
            x -= distance
        elif (move[0] == 'R'):
            distance = int(move[1:len(move)])
            sections.append(((x,y), (x+distance, y)))
            x += distance
        elif (move[0] == 'U'):
            distance = int(move[1:len(move)])
            sections.append(((x,y), (x, y+distance)))
            y += distance
        elif (move[0] == 'D'):
            distance = int(move[1:len(move)])
            sections.append(((x,y), (x, y-distance)))
            y -= distance
            
    return sections
        

In [110]:
def get_wire_as_object(wire):
    x = 0
    y = 0
    steps = 0
    sections = []
    for move in wire:
        distance = int(move[1:len(move)])
        steps += distance
        if (move[0] == 'L'):
            sections.append(Wire(x, y, x-distance, y, steps))
            x -= distance
        elif (move[0] == 'R'):
            sections.append(Wire(x, y, x+distance, y, steps))
            x += distance
        elif (move[0] == 'U'):
            sections.append(Wire(x, y, x, y+distance, steps))
            y += distance
        elif (move[0] == 'D'):
            sections.append(Wire(x, y, x, y-distance, steps))
            y -= distance
            
    return sections
        

In [92]:
def get_crosses(wire, wires):
    closest = 1000000
    for crossed_wire in wires:
        if (are_wires_crossed(wire, crossed_wire)):
            distance = manhattan_distance(wire.start.y, crossed_wire.start.x)
            if (distance < closest):
                closest = distance
        elif (are_wires_crossed(crossed_wire, wire)):
            distance = manhattan_distance(wire.start.x, crossed_wire.start.y)
            if (distance < closest):
                closest = distance
                
    return closest
        

In [126]:
def get_crosses_as_steps(wire, wires):
    closest = 1000000
    for crossed_wire in wires:
        if (are_wires_crossed(wire, crossed_wire)):
            distance = wire.steps + crossed_wire.steps - abs(crossed_wire.end.y - wire.end.y) - abs(wire.end.x - crossed_wire.end.x)
            if (distance < closest):
                closest = distance
        elif (are_wires_crossed(crossed_wire, wire)):
            distance = wire.steps + crossed_wire.steps - abs(crossed_wire.end.x - wire.end.x) - abs(wire.end.y - crossed_wire.end.y)
            if (distance < closest):
                closest = distance
        
                
    return closest
        

In [116]:
def get_steps(wire, wires):
    for crossed_wire in wires:
        if (are_wires_crossed(wire, crossed_wire) or are_wires_crossed(crossed_wire, wire)):
            steps = min(wire.steps, crossed_wire.steps)
            wire.set_steps(steps)
            crossed_wire.set_steps(steps)
        
                   
        

In [80]:
def are_wires_crossed(lwire, rwire):
    return arebetween(lwire.start.y, lwire.end.y, rwire.start.y, rwire.end.y) and arebetween(rwire.start.x, rwire.end.x, lwire.start.x, lwire.end.x)

In [81]:
def manhattan_distance(x, y):
    return abs(x) + abs(y)

In [96]:
wires = list(map(get_wire_as_object, data))
closest = 100000
for wire in wires[0]:
    value = get_crosses(wire, wires[1])
    if (value < closest):
        closest = value
    
print(closest)
    

293


In [128]:
wires = list(map(get_wire_as_object, data))
closest = 100000
##for wire in wires:
##    for step in wire:
##        get_steps(step, wire)
for wire in wires[0]:
    value = get_crosses_as_steps(wire, wires[1])
    if (value < closest):
        closest = value
    
print(closest)
    

27306


In [101]:
wires = list(map(get_wire_as_object, data))
for wire in wires[0]:
    if wire.steps > 0:
        print(wire.steps)

In [24]:
def isbetween (x, a, b):
    return a < x < b or a > x > b

In [84]:
def arebetween (x, y, a, b):
    return isbetween(x, a, b) and isbetween(y, a, b)

In [ ]:
def fill_steps(wires):
    for wire in wires:
        get_crosses(wire, wires)